In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter

In [ ]:
data = pd.read_csv('C://Users//А4//projects//Projects//module_4//data project 4.csv')
data.sample(5)

In [16]:
data.describe()

,flight_id,range,total_amount,count_seat_economy,count_seat_business,total_amount_business
count,193.000000,193.000000,1.800000e+02,193.000000,193.0,180.000000
mean,136540.466321,3640.414508,1.179647e+06,102.611399,12.0,297346.666667
std,249.389171,600.194269,4.811836e+05,16.505342,0.0,96237.646613
min,136119.000000,3000.000000,5.310000e+05,85.000000,12.0,113400.000000
25%,136320.000000,3000.000000,7.385250e+05,85.000000,12.0,226800.000000
50%,136534.000000,4200.000000,1.068850e+06,118.000000,12.0,226800.000000
75%,136768.000000,4200.000000,1.631950e+06,118.000000,12.0,402600.000000
max,136961.000000,4200.000000,1.886000e+06,118.000000,12.0,439200.000000


In [17]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 193 entries, 0 to 192
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   flight_id              193 non-null    int64  
 1   flight_no              193 non-null    object 
 2   departure_airport      193 non-null    object 
 3   arrival_airport        193 non-null    object 
 4   model                  193 non-null    object 
 5   actual_departure       193 non-null    object 
 6   actual_arrival         193 non-null    object 
 7   range                  193 non-null    int64  
 8   total_amount           180 non-null    float64
 9   count_seat_economy     193 non-null    int64  
 10  count_seat_business    193 non-null    int64  
 11  total_amount_business  180 non-null    float64
dtypes: float64(2), int64(4), object(6)
memory usage: 18.2+ KB


In [18]:
# при просмотре таблицы видно, что для Новокузнецка нет данных total_amount
pd.isnull(data).sum()

flight_id                 0
flight_no                 0
departure_airport         0
arrival_airport           0
model                     0
actual_departure          0
actual_arrival            0
range                     0
total_amount             13
count_seat_economy        0
count_seat_business       0
total_amount_business    13
dtype: int64

In [19]:
# удаляем строки с нулевым значением в столбце total_amount
data1 = data.dropna(subset=['total_amount'])
pd.isnull(data1).sum()

flight_id                0
flight_no                0
departure_airport        0
arrival_airport          0
model                    0
actual_departure         0
actual_arrival           0
range                    0
total_amount             0
count_seat_economy       0
count_seat_business      0
total_amount_business    0
dtype: int64

In [20]:
data1.pop('range')
data1.head(2)

,flight_id,flight_no,departure_airport,arrival_airport,model,actual_departure,actual_arrival,total_amount,count_seat_economy,count_seat_business,total_amount_business
0,136119,PG0252,AAQ,SVO,Boeing 737-300,2017-01-03T10:08:00Z,2017-01-03T11:49:00Z,1653000.0,118,12,402600.0
1,136120,PG0252,AAQ,SVO,Boeing 737-300,2017-02-24T10:07:00Z,2017-02-24T11:46:00Z,1605400.0,118,12,402600.0


In [ ]:
# Предположим, что Цена авиационного топлива за тонну 2017
# https://regionneftesbit.ru/cena-aviacionnogo-topliva-za-tonnu-2017/
# 37 рублей за 1 кг или 37000 рублей за тонну   
# Цену в модели можно уточнить, если найдется дополнение

In [26]:
id_list = ["1"]
data1['fuel_cost'] = np.repeat(id_list, 180)
data1.head(1)

<ipython-input-26-2b6a271e125f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1['fuel_cost'] = np.repeat(id_list, 180)


,flight_id,flight_no,departure_airport,arrival_airport,model,actual_departure,actual_arrival,total_amount,count_seat_economy,count_seat_business,total_amount_business,fuel_cost
0,136119,PG0252,AAQ,SVO,Boeing 737-300,2017-01-03T10:08:00Z,2017-01-03T11:49:00Z,1653000.0,118,12,402600.0,1


In [31]:
# преобразуем столбец fuel_cost в числовой формат 
data1['fuel_cost'] = data1['fuel_cost'].apply(pd.to_numeric)

<ipython-input-31-371eea92d64e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1['fuel_cost'] = data1['fuel_cost'].apply(pd.to_numeric)


In [32]:
data1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 180 entries, 0 to 192
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   flight_id              180 non-null    int64  
 1   flight_no              180 non-null    object 
 2   departure_airport      180 non-null    object 
 3   arrival_airport        180 non-null    object 
 4   model                  180 non-null    object 
 5   actual_departure       180 non-null    object 
 6   actual_arrival         180 non-null    object 
 7   total_amount           180 non-null    float64
 8   count_seat_economy     180 non-null    int64  
 9   count_seat_business    180 non-null    int64  
 10  total_amount_business  180 non-null    float64
 11  fuel_cost              180 non-null    int64  
dtypes: float64(2), int64(4), object(6)
memory usage: 18.3+ KB


In [5]:
# Для SSJ, минимальный часовой расход обеспечивается на М = 0.75, а вот минимальный километровый — на М = 0.78…0.79. 
# http://superjet.wikidot.com/wiki:rashod-topliva
# https://aviation21.ru/sukhoj-superjet-100/
# Итого, за январь: 1,670 кг/ч